<a href="https://colab.research.google.com/github/shiz-107/Elliptic-Isogenies-PQC/blob/main/4_steps_on_montgomery_EC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# for required torsion group , parameters of curves
j = complex(0, 1)
p = 7
b = 2+1j
for s in range(p):
    a = 2 + s * j
    for t in range(p):
        for u in range(p):
            c = t + u * j
            count = 0  # Counter variable to keep track of the points

            for m in range(p):
                for n in range(p):
                    i = m + n * j
                    Z = (i ** 3 + a * i ** 2 + b*i + c)  # Removed the redundant '1'
                    X = complex(Z.real % p, Z.imag % p)  # Simplified the assignment

                    for k_m in range(p):
                        for k_n in range(p):
                            k = k_m + k_n * j
                            Y = complex((k ** 2).real % p, (k ** 2).imag % p)
                            if X == Y:
                                R = (i, k)
                                count += 1  # Increment the counter
                                break  # Added the 'break' statement here to exit the loop when a match is found
                        else:
                            continue
                        break

            if count ==33:
                print(f"s = {s}, c = {t+u*j}, Count = {count}")  # Corrected the 'uj' to 'u*j' in the print statement


s = 6, c = (1+6j), Count = 33


In [ ]:
j = complex(0, 1)
p = 7
a = 2+1j
b = 0+6j
count = 0  # Counter variable to keep track of the points

for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i**3 + a*i**2 + i +b)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j

        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j

                z = (k ** 2)
                real_part = z.real % p
                imaginary_part = z.imag % p
                Y = real_part + imaginary_part * 1j

                if X == Y:
                    R = (i, k)
                    print(R)
                    count += 1  # Increment the counter

print( count)

In [ ]:
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []

for m in range(p):
    for n in range(p):
        i = m + n * j

        Z = (i ** 3 + a * i ** 2 + b*i * 1 +c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j

        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j

                z = (k ** 2)
                real_part = z.real % p
                imaginary_part = z.imag % p
                Y = real_part + imaginary_part * 1j

                if X == Y:
                    R.append((i, k))

# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope**2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

# Define a function to compute the order of a point on the curve
def point_order(P):
    if P is None:
        return 1
    Q = P
    order = 1
    while Q is not None:
        Q = point_addition(P, Q)
        order = order + 1
        if order > p + 1:
            return None
    return order

def find_points_with_order(order):
    points = []
    for P in R:
        P_order = point_order(P)
        if P_order == order:
            points.append(P)
    return points

target_order = 2
points_with_order = find_points_with_order(target_order)

print(f"Points with order {target_order}:")
for P in points_with_order:
    print(P)

Points with order 2:
(6j, (4+6j))
(6j, (9+7j))
((1+1j), (1+8j))
((1+1j), (12+5j))
((1+6j), (4+7j))
((1+6j), (9+6j))
((2+11j), (2+10j))
((2+11j), (11+3j))
((3+3j), (2+10j))
((3+3j), (11+3j))
((3+4j), (6+9j))
((3+4j), (7+4j))
((5+0j), (6+4j))
((5+0j), (7+9j))
((6+5j), (1+5j))
((6+5j), (12+8j))
((7+10j), 0j)
((8+5j), (1+8j))
((8+5j), (12+5j))
((9+0j), (2+3j))
((9+0j), (11+10j))
((10+12j), (3+2j))
((10+12j), (10+11j))
((11+3j), (5+1j))
((11+3j), (8+12j))
((11+4j), (2+10j))
((11+4j), (11+3j))


In [ ]:
# poins doubling , addition , order, nP, order chk
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
infinity = None

# Define a function to add two points on the curve
def point_addition(P, Q):
    # Point at infinity
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope ** 2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

P =
Q = P
point_addition(P,Q)
for i in range(121):
  Q=point_addition(P,Q)
  print(Q)

((12+4j), (7+12j))
((2+12j), (6+12j))
((7+2j), 1j)
((1+5j), (11+1j))
((4+1j), 7j)
((3+3j), (11+3j))
((9+0j), (2+3j))
((7+9j), (7+11j))
((8+7j), (2+9j))
((6+6j), (3+7j))
((8+10j), (8+11j))
((7+0j), (2+0j))


TypeError: ignored

In [ ]:
# Points addition
# Define the parameters of the elliptic curve E : y^2=x^3+Ax+x over F_431[j]
j = complex(0,1)
a = 3+2j
p = 431 # the field size


# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P



    if P != Q:

        num = Q[1] - P[1]
        num_prime = num.real%p + num.imag%p*j

        den = Q[0] - P[0]
        den_prime = den.real%p + den.imag%p*j
        conj_den = den_prime.real%p + (p-den_prime.imag)%p*j
        ab = ((den_prime.real)**2+(den_prime.imag)**2)%p

        def find_inverse(element, mod):
          if element == 0:
            return None

          # Apply the extended Euclidean algorithm
          a, b = mod, element
          x, y = 0, 1
          while b != 0:
               quotient = a // b
               a, b = b, a - quotient * b
               x, y = y, x - quotient * y

           # Ensure the inverse is positive
          inverse = x % mod
          if inverse < 0:
               inverse += mod

          return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)
        print(inverse)
        den_double = conj_den*inverse
        den_triple = den_double.real%p + den_double.imag%p*j
        slope_prime = num_prime*den_triple
        slope = slope_prime.real%p + slope_prime.imag%p*j
        print(slope)


    Rx = slope**2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real%p , Rx.imag%p)
    print(Rx_prime)
    Ry = -(P[1] + slope * (Rx_prime - P[0]) )
    Ry_prime = complex(Ry.real%p , Ry.imag%p)
    return (Rx_prime, Ry_prime)



# Test point addition
P=(1j, (97+150j))
Q=(2j, (209+396j))
R=point_addition(P,Q)
print(R)

In [ ]:
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
P = ((12+12j), (1+1j))
f = 3 * P[0] ** 2 + 2 * a * P[0] + b
f_prime = (f.real % p) + (f.imag % p) * j
print(f"P = {P}, f(P) = {f_prime}")
g = -2 * P[1]
g_prime = (g.real % p) + (g.imag % p) * j
print(f"P = {P}, g(P) = {g_prime}")

P = ((12+12j), (1+1j)), f(P) = (1+5j)
P = ((12+12j), (1+1j)), g(P) = (11+11j)


In [ ]:
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = i ** 3 + a * i ** 2 + b*i + c
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  count+=1
                  break
            else:
                    continue
            break
for P in R:
    f = 3 * P[0] ** 2 + 2 * a * P[0] + b
    f_prime = (f.real % p) + (f.imag % p) * j
    g = -2 * P[1]
    g_prime = (g.real % p) + (g.imag % p) * j
    print(f"P = {P},f(P) = {f_prime}, g(p) = {g_prime}")
print(count)


P = (6j, (4+6j)),f(P) = (11+2j), g(p) = (5+1j)
P = (7j, 6j),f(P) = (9+4j), g(p) = 1j
P = (9j, 3j),f(P) = 8j, g(p) = 7j
P = ((1+1j), (1+8j)),f(P) = (1+0j), g(p) = (11+10j)
P = ((1+5j), (2+12j)),f(P) = (12+6j), g(p) = (9+2j)
P = ((1+6j), (4+7j)),f(P) = (3+1j), g(p) = (5+12j)
P = ((1+7j), 5j),f(P) = (1+9j), g(p) = 3j
P = ((2+1j), (3+12j)),f(P) = (12+8j), g(p) = (7+2j)
P = ((2+10j), (3+4j)),f(P) = (9+4j), g(p) = (7+5j)
P = ((2+11j), (2+10j)),f(P) = (9+5j), g(p) = (9+6j)
P = ((2+12j), (5+4j)),f(P) = (3+6j), g(p) = (3+5j)
P = ((3+3j), (2+10j)),f(P) = (1+4j), g(p) = (9+6j)
P = ((3+4j), (6+9j)),f(P) = (4+11j), g(p) = (1+8j)
P = ((3+6j), (1+2j)),f(P) = (5+12j), g(p) = (11+9j)
P = ((3+8j), (1+4j)),f(P) = (8+0j), g(p) = (11+5j)
P = ((3+9j), (2+4j)),f(P) = (7+7j), g(p) = (9+5j)
P = ((4+0j), (2+9j)),f(P) = (5+11j), g(p) = (9+8j)
P = ((4+1j), 6j),f(P) = 11j, g(p) = 1j
P = ((4+3j), (2+7j)),f(P) = (11+11j), g(p) = (9+12j)
P = ((4+11j), (1+2j)),f(P) = (10+11j), g(p) = (11+9j)
P = ((5+0j), (6+4j)),f(P) 

In [ ]:
#  points except the pounts of target order  (a means c) and (b means d)
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 +b* i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  break
            else:
                  continue
            break
# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope ** 2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

# Define a function to compute the order of a point on the curve
def point_order(P):
    if P is None:
        return 1
    Q = P
    order = 1
    while Q is not None:
        Q = point_addition(P, Q)
        order = order + 1
        if order > p + 1:
            return None
    return order

def find_points_with_order(order):
    points = []
    for P in R:
        P_order = point_order(P)
        if P_order == order:
            points.append(P)
    return points

target_order = 2
points_with_order = find_points_with_order(target_order)

print(f"Points with order not {target_order}:")
for P in R:
    if P not in points_with_order:
       count+=1
print(count)
f_prime_list = []
for P in R:
    if P not in points_with_order:
        f = 2*(3*P[0]**2 + 2*a*P[0] + b)      # 2f
        f_prime = complex(f.real % p, f.imag % p)
        print(f_prime)
        f_prime_list.append(f_prime)
total_sum = complex(0, 0)
for f_prime in f_prime_list:
    total_sum += f_prime.real
    v = (total_sum.real)%p +  (total_sum.imag %p)*j
print(v)

Points with order not 2:
42
(5+8j)
3j
(11+12j)
(2+5j)
(11+3j)
(5+8j)
(6+12j)
(10+11j)
(3+0j)
(1+1j)
(10+9j)
9j
(9+9j)
(7+9j)
(10+7j)
(3+5j)
(1+1j)
9j
(2+5j)
(8+1j)
(12+8j)
(6+2j)
(11+12j)
(10+7j)
(9+1j)
(5+10j)
(2+6j)
(12+11j)
(11+6j)
(11+12j)
(2+8j)
(5+7j)
(8+4j)
(2+6j)
(4+2j)
(8+1j)
(3+0j)
(7+7j)
(11+12j)
(9+9j)
(1+0j)
(2+10j)
(8+0j)


In [ ]:
j = complex(0, 1)
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 + b*i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  break
            else:
                  continue
            break
# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope ** 2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

# Define a function to compute the order of a point on the curve
def point_order(P):
    if P is None:
        return 1
    Q = P
    order = 1
    while Q is not None:
        Q = point_addition(P, Q)
        order = order + 1
        if order > p + 1:
            return None
    return order

def find_points_with_order(order):
    points = []
    for P in R:
        P_order = point_order(P)
        if P_order == order:
            points.append(P)
    return points
f_prime_list = []
target_order = 2
points_with_order = find_points_with_order(target_order)
print("points_with_order:" , points_with_order)
for P in R:
    if P not in points_with_order:
        f = 2*(3*P[0]**2 + 2*a*P[0] + b)      # 2f
        f_prime = complex(f.real % p, f.imag % p)
        f_prime_list.append(f_prime)
total_sum = complex(0, 0)
for f_prime in f_prime_list:
    total_sum += f_prime
    v = (total_sum.real)%p +  (total_sum.imag %p)*j
print(v)

points_with_order: [(6j, (4+6j)), ((1+1j), (1+8j)), ((1+6j), (4+7j)), ((2+11j), (2+10j)), ((3+3j), (2+10j)), ((3+4j), (6+9j)), ((5+0j), (6+4j)), ((6+5j), (1+5j)), ((7+10j), 0j), ((8+5j), (1+8j)), ((9+0j), (2+3j)), ((10+12j), (3+2j)), ((11+3j), (5+1j)), ((11+4j), (2+10j))]
(8+8j)


In [ ]:

j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 + b*i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  break
            else:
                  continue
            break
# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope ** 2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

# Define a function to compute the order of a point on the curve
def point_order(P):
    if P is None:
        return 1
    Q = P
    order = 1
    while Q is not None:
        Q = point_addition(P, Q)
        order = order + 1
        if order > p + 1:
            return None
    return order

def find_points_with_order(order):
    points = []
    for P in R:
        P_order = point_order(P)
        if P_order == order:
            points.append(P)
    return points
f_prime_list = []
f_o2_list = []
target_order = 2
points_with_order = find_points_with_order(target_order)
print("points_with_order_2:" , points_with_order)
for P in points_with_order:
  f_o2 = 3*P[0]**2 + 2*a*P[0] + b
  f_o2_prime= complex(f_o2.real % p, f_o2.imag % p)
  f_o2_list.append(f_o2_prime)
  print(f_o2_prime)
for P in R:
    if P not in points_with_order:
        f = 2*(3*P[0]**2 + 2*a*P[0] + b)      # 2f
        f_prime = complex(f.real % p, f.imag % p)
        f_prime_list.append(f_prime)
total_sum = complex(0, 0)
for f_prime in f_prime_list:
    total_sum += f_prime
    v = (total_sum.real)%p +  (total_sum.imag %p)*j
print("v:",v)
totall_sum = complex(0, 0)
for f_o2_prime in f_o2_list:
  totall_sum += f_o2_prime
  u = (totall_sum.real)%p +  (totall_sum.imag %p)*j
print("u:",u)
final = v+u
print("final:" ,final)

points_with_order_2: [(6j, (4+6j)), ((1+1j), (1+8j)), ((1+6j), (4+7j)), ((2+11j), (2+10j)), ((3+3j), (2+10j)), ((3+4j), (6+9j)), ((5+0j), (6+4j)), ((6+5j), (1+5j)), ((7+10j), 0j), ((8+5j), (1+8j)), ((9+0j), (2+3j)), ((10+12j), (3+2j)), ((11+3j), (5+1j)), ((11+4j), (2+10j))]
(11+2j)
(1+0j)
(3+1j)
(9+5j)
(1+4j)
(4+11j)
(8+0j)
(10+10j)
(11+2j)
(7+9j)
(2+8j)
(8+0j)
(2+8j)
(5+11j)
v: (8+8j)
u: (4+6j)
final: (12+14j)


In [ ]:
j = complex(0, 1)
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 + b*i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  count+=1
                  break
            else:
                    continue
            break
g_list = []
for P in R:
    g = -2 * P[1]
    g_prime = (g.real % p) + (g.imag % p) * j
    g_dou_prime = (g_prime)**2
    g_triple_prime = g_dou_prime.real%p + (g_dou_prime.imag%p)*j
    g_list.append(g_triple_prime)
    print(f"g_prime={g_prime},g_dou-prime ={g_dou_prime} ,g_triple_prime={g_triple_prime}")

total_sum = complex(0, 0)
for g_triple_prime in g_list:
    total_sum += g_triple_prime
    u = total_sum.real%p + (total_sum.imag%p)*j
print("u:" , u)


g_prime=(5+1j),g_dou-prime =(24+10j) ,g_triple_prime=(11+10j)
g_prime=1j,g_dou-prime =(-1+0j) ,g_triple_prime=(12+0j)
g_prime=7j,g_dou-prime =(-49+0j) ,g_triple_prime=(3+0j)
g_prime=(11+10j),g_dou-prime =(21+220j) ,g_triple_prime=(8+12j)
g_prime=(9+2j),g_dou-prime =(77+36j) ,g_triple_prime=(12+10j)
g_prime=(5+12j),g_dou-prime =(-119+120j) ,g_triple_prime=(11+3j)
g_prime=3j,g_dou-prime =(-9+0j) ,g_triple_prime=(4+0j)
g_prime=(7+2j),g_dou-prime =(45+28j) ,g_triple_prime=(6+2j)
g_prime=(7+5j),g_dou-prime =(24+70j) ,g_triple_prime=(11+5j)
g_prime=(9+6j),g_dou-prime =(45+108j) ,g_triple_prime=(6+4j)
g_prime=(3+5j),g_dou-prime =(-16+30j) ,g_triple_prime=(10+4j)
g_prime=(9+6j),g_dou-prime =(45+108j) ,g_triple_prime=(6+4j)
g_prime=(1+8j),g_dou-prime =(-63+16j) ,g_triple_prime=(2+3j)
g_prime=(11+9j),g_dou-prime =(40+198j) ,g_triple_prime=(1+3j)
g_prime=(11+5j),g_dou-prime =(96+110j) ,g_triple_prime=(5+6j)
g_prime=(9+5j),g_dou-prime =(56+90j) ,g_triple_prime=(4+12j)
g_prime=(9+8j),g_dou-prime =(

In [ ]:
j = complex(0, 1)
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 + b*i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  count+=1
                  break
            else:
                    continue
            break
for P in R:
    f = (3*P[0]**2 + 2*a*P[0] + b)*P[0]
    f_prime = (f.real % p) + (f.imag % p) * j
    print(f_prime)


(1+1j)
(11+11j)
(6+0j)
(1+1j)
(8+1j)
(10+6j)
(3+3j)
(3+2j)
(4+7j)
(2+5j)
(12+9j)
(4+2j)
(7+10j)
(8+1j)
(11+12j)
(10+6j)
(7+5j)
(2+5j)
(11+12j)
(10+11j)
(1+0j)
(4+2j)
(7+5j)
(3+2j)
(8+1j)
(10+6j)
(4+8j)
(7+9j)
(3+2j)
(6+0j)
(1+0j)
(10+11j)
(5+7j)
1j
(11+3j)
(11+12j)
(10+6j)
(6+0j)
(5+7j)
(11+12j)
(7+9j)
9j
(12+9j)
9j
(3+2j)
(2+5j)
(11+3j)
(11+11j)
(4+8j)
(8+1j)
(10+7j)
(6+0j)
(2+5j)
(10+7j)
(6+5j)
(4+7j)


In [ ]:
j = complex(0, 1)
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
R = []
count = 0
for m in range(p):
    for n in range(p):
        i = m + n * j
        Z = (i ** 3 + a * i ** 2 + b*i + c)
        real_part = Z.real % p
        imaginary_part = Z.imag % p
        X = real_part + imaginary_part * 1j
        for k_m in range(p):
            for k_n in range(p):
                k = k_m + k_n * j
                Y = (k ** 2).real % p + ((k ** 2).imag % p) * j
                if X == Y:
                  R.append((i, k))
                  break
            else:
                  continue
            break
# Define a function to add two points on the curve
def point_addition(P, Q):
    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        num = 3 * P[0] ** 2 + 2 * a * P[0] + b
        num_prime = complex(num.real % p, num.imag % p)
        den = complex(2 * P[1].real % p, 2 * P[1].imag % p)
        conj_den = den.real % p + (p - den.imag) % p * j
        ab = ((den.real) ** 2 + (den.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_prime = complex((conj_den * inverse).real % p, (conj_den * inverse).imag % p)
        slope = complex((num_prime * den_prime).real % p, (num_prime * den_prime).imag % p)

    if P != Q:
        num = Q[1] - P[1]
        num_prime = num.real % p + num.imag % p * j

        den = Q[0] - P[0]
        den_prime = den.real % p + den.imag % p * j
        conj_den = den_prime.real % p + (p - den_prime.imag) % p * j
        ab = ((den_prime.real) ** 2 + (den_prime.imag) ** 2) % p

        def find_inverse(element, mod):
            if element == 0:
                return None

            # Apply the extended Euclidean algorithm
            a, b = mod, element
            x, y = 0, 1
            while b != 0:
                quotient = a // b
                a, b = b, a - quotient * b
                x, y = y, x - quotient * y

            # Ensure the inverse is positive
            inverse = x % mod
            if inverse < 0:
                inverse += mod

            return inverse

        # Test the function
        element = ab
        mod = p
        inverse = find_inverse(element, mod)

        if inverse is None:
            return None

        den_double = conj_den * inverse
        den_triple = den_double.real % p + den_double.imag % p * j
        slope_prime = num_prime * den_triple
        slope = slope_prime.real % p + slope_prime.imag % p * j

    Rx = slope ** 2 - a - P[0] - Q[0]
    Rx_prime = complex(Rx.real % p, Rx.imag % p)
    Ry = -(P[1] + slope * (Rx_prime - P[0]))
    Ry_prime = complex(Ry.real % p, Ry.imag % p)
    return (Rx_prime, Ry_prime)

# Define a function to compute the order of a point on the curve
def point_order(P):
    if P is None:
        return 1
    Q = P
    order = 1
    while Q is not None:
        Q = point_addition(P, Q)
        order = order + 1
        if order > p + 1:
            return None
    return order

def find_points_with_order(order):
    points = []
    for P in R:
        P_order = point_order(P)
        if P_order == order:
            points.append(P)
    return points
f_prime_list = []
f_o2_list = []
target_order = 2
points_with_order = find_points_with_order(target_order)
print("points_with_order_2:" , points_with_order)
for P in points_with_order:
  f_o2 = (3*P[0]**2 + 2*a*P[0] + b)*P[0]
  f_o2_prime= complex(f_o2.real % p, f_o2.imag % p)
  f_o2_list.append(f_o2_prime)
  print(f_o2_prime)
for P in R:
    if P not in points_with_order:
        f = 2*(3*P[0]**2 + 2*a*P[0] + b) *P[0]     # 2f
        f_prime = complex(f.real % p, f.imag % p)
        f_prime_list.append(f_prime)
total_sum = complex(0, 0)
for f_prime in f_prime_list:
    total_sum += f_prime
    v = (total_sum.real)%p +  (total_sum.imag %p)*j
print("v:",v)
totall_sum = complex(0, 0)
for f_o2_prime in f_o2_list:
  totall_sum += f_o2_prime
  u = (totall_sum.real)%p +  (totall_sum.imag %p)*j
print("u:",u)
final = v+u
print("final:" ,final)
g_list = []
for P in R:
    g = (-2*P[1])
    g_prime = (g.real % p) + (g.imag % p) * j
    g_dou_prime = (g_prime)**2
    g_triple_prime = g_dou_prime.real%p + (g_dou_prime.imag%p)*j
    g_list.append(g_triple_prime)
    print(f"g_prime={g_prime},g_dou-prime ={g_dou_prime} ,g_triple_prime={g_triple_prime}")

totalll_sum = complex(0, 0)
for g_triple_prime in g_list:
    totalll_sum += g_triple_prime
    u = totalll_sum.real%p + (totalll_sum.imag%p)*j
print("u:" , u)
print(final+u)

points_with_order_2: [(6j, (4+6j)), ((1+1j), (1+8j)), ((1+6j), (4+7j)), ((2+11j), (2+10j)), ((3+3j), (2+10j)), ((3+4j), (6+9j)), ((5+0j), (6+4j)), ((6+5j), (1+5j)), ((7+10j), 0j), ((8+5j), (1+8j)), ((9+0j), (2+3j)), ((10+12j), (3+2j)), ((11+3j), (5+1j)), ((11+4j), (2+10j))]
(1+1j)
(1+1j)
(10+6j)
(2+5j)
(4+2j)
(7+10j)
(1+0j)
(10+6j)
(5+7j)
(11+3j)
(5+7j)
(2+5j)
(11+3j)
(11+11j)
v: (5+0j)
u: (3+2j)
final: (8+2j)
g_prime=(5+1j),g_dou-prime =(24+10j) ,g_triple_prime=(11+10j)
g_prime=1j,g_dou-prime =(-1+0j) ,g_triple_prime=(12+0j)
g_prime=7j,g_dou-prime =(-49+0j) ,g_triple_prime=(3+0j)
g_prime=(11+10j),g_dou-prime =(21+220j) ,g_triple_prime=(8+12j)
g_prime=(9+2j),g_dou-prime =(77+36j) ,g_triple_prime=(12+10j)
g_prime=(5+12j),g_dou-prime =(-119+120j) ,g_triple_prime=(11+3j)
g_prime=3j,g_dou-prime =(-9+0j) ,g_triple_prime=(4+0j)
g_prime=(7+2j),g_dou-prime =(45+28j) ,g_triple_prime=(6+2j)
g_prime=(7+5j),g_dou-prime =(24+70j) ,g_triple_prime=(11+5j)
g_prime=(9+6j),g_dou-prime =(45+108j) ,g_trip

In [ ]:
j = complex(0, 1)
p = 13
a = 1+1j
b = 1+3j
c = 8+8j
v = 12+14j
w = 0+12j
B = b-5*v
C = c - 4*a*v -7*w
B_prime = complex(B.real%p , B.imag%p)
C_prime = complex(C.real%p , C.imag%p )
print(B_prime)
print(C_prime)

(6+11j)
(3+2j)
